In [47]:
import os
from tqdm import tqdm
import csv
import random
import llm_client
from AudiosetDataset import AudiosetDataset
import torch
import json
import requests

# print(os.chdir("/home/junchenl/diffusers_with_dataloader/examples/text_to_image/audio_gen_files"))

In [45]:
local = "/home/junchenl/AudioTaggingDoneRight/egs/audioset/data"
label_csv=os.path.join(local, 'class_labels_indices.csv')
data_path = os.path.join(local,'datafiles/audioset_bal_unbal_train_data.json')

alpaca_client = llm_client.Client(address="tir-1-7")
llama_client  = llm_client.Client(address="tir-0-15")

In [14]:
data_file = open(data_path)

data = json.load(data_file)


In [44]:
label_dict = {}
with open(label_csv,'r') as data_file:
    for line in csv.reader(data_file):
        label_dict[line[1]] = line[2]
        


In [21]:

def gen_caption(classes, client):
    human_sample = ", ".join(classes[:-2] + [", and ".join(classes[-2:])])
    prompt = ""
    prompt += "For each of the 3 line summarize the sounds into a single sentence: \n"
    prompt += "imagine a scene with all of these sounds existing together: (inside, small room)." + "\n > This is happening inside a small room. \n\n"
    prompt += "imagine a scene with all of these sounds existing together: (dog, bark, howl, speech)." + "\n > a dog barking and howling with a person speaking aswell. \n\n"
    prompt += "imagine a scene with all of these sounds existing together: " + f"({human_sample}). \n >"
    outputs = client.prompt(prompt)
    outputs = outputs[0].text[len(prompt) + 1:]
    final_output = outputs.split("\n")[0]
    return final_output

In [33]:
import time

In [46]:
dataset = [
    "here is a cookie recipe: ",
    "here is a brownie recipe: ",
    "here is a sandiwch recipe: ",
    "here is a cookie cake recipe: "
]
print("running list")
pre = time.time()
for _ in range(5):
    list_gen = alpaca_client.prompt(dataset)
post = time.time()

list_dt = post - pre

print("running ind")
pre = time.time()
ind_gen = [0,0,0,0,0,0,0,0]
for _ in range(5):
    for i in range(4):
        ind_gen[i] = alpaca_client.prompt(dataset[i])
post = time.time()

ind_dt = post - pre


print("running comp")
pre = time.time()
for _ in range(5):
    comp_gen = [alpaca_client.prompt(p) for p in dataset]
post = time.time()

comp_dt = post - pre


print("LIST: ", list_dt)
print("INDEX: ", ind_dt)
print("COMP: ", comp_dt)

running list


ServerError: Server-side Error -- RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [31]:
answer = alpaca_client.prompt(["here is a cookie recipe: ", "here is a sandwich recipe: "])

In [32]:
answer

[Output(text=' here is a cookie recipe: 1 cup of butter, 2 cups of sugar, 3 eggs, 1 teaspoon of vanilla extract, 1 and 1/4 cups of all-purpose flour, 1 teaspoon of salt, 1 cup of chocolate chips,', scores=None, hidden_states=None),
 Output(text=' here is a sandwich recipe: 1 slice whole wheat bread, 2 slices roast beef, 2 slices melted cheddar cheese, 1 tomato slice, 1 leaf of lettuce, 1 tbsp of horseradish sauce, and 1 tbsp of wasabi', scores=None, hidden_states=None)]

In [50]:
    
# for i in tqdm(range(len(data["data"]))):

batch_size = 3

for i in tqdm(range(0, 12, batch_size)):
    for j in range(i, i + batch_size):
        print(i,j)
    datum = data["data"][i]
    labels = datum['labels'].split(",")
    labels = [label_dict[label] for label in labels]
    
    cap = gen_caption(labels, alpaca_client)
    
    datum["caption"] = cap
    datum["classes"] = labels
    
print("Done")

  0%|                                                                                                                        | 0/4 [00:00<?, ?it/s]

0 0
0 1
0 2


 25%|████████████████████████████                                                                                    | 1/4 [00:01<00:03,  1.22s/it]

3 3
3 4
3 5


 50%|████████████████████████████████████████████████████████                                                        | 2/4 [00:02<00:02,  1.08s/it]

6 6
6 7
6 8


 75%|████████████████████████████████████████████████████████████████████████████████████                            | 3/4 [00:02<00:00,  1.29it/s]

9 9
9 10
9 11


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:03<00:00,  1.26it/s]

Done


In [ ]:
output_path = os.path.join(local,'datafiles/alpaca_audioset_bal_unbal_train_data.json')

out_file = open(output_path, "w")
json.dump({"data": data["data"]}, out_file)

In [69]:
test_file = open("ids_with_captions.json", "r")


test_data = json.load(test_file)["data"]

print(os.chdir("/home/junchenl/diffusers_with_dataloader/examples/text_to_image/audio_gen_files"))



In [121]:
top = len(test_data)
n_samples = 3

rand_ids = [random.randint(0, top) for _ in range(n_samples)]

name_list = list(test_data.keys())

sample = [test_data[name_list[idx]] for idx in rand_ids]
ids = [name_list[idx] for idx in rand_ids]
print(sample)

['Vehicle, Vehicle horn, car horn, honking, Speech', 'Music, Chatter, Speech, Inside, small room', 'Domestic animals, pets, Speech, Dog, Animal, Outside, rural or natural']


In [123]:

for sam, idx in zip(sample, ids):
    ls_sam  = sam.split(", ")
    human_sample = ", ".join(ls_sam[:-2] + [", and ".join(ls_sam[-2:])])

    prompt = "imagine a scene with all of these sounds existing together: " + human_sample + ". Then describe this scene in one sentence."
    print("ID: ", idx)
    print("PROMPT: ", prompt, "\n") 
    outputs = llama_client.prompt(prompt)
    # outputs = alpaca_client.prompt(prompt)
    print(outputs[0].text)
    print("=================================================")

ID:  N-J89jHemZg
PROMPT:  imagine a scene with all of these sounds existing together: Vehicle, Vehicle horn, car horn, honking, and Speech. Then describe this scene in one sentence. 

 imagine a scene with all of these sounds existing together: Vehicle, Vehicle horn, car horn, honking, and Speech. Then describe this scene in one sentence. In one minute describe the situation where all of these sounds co-exist.
Bike and car with all types of people around it, pedestrians waiting to cross on street light.
A horn from an angry driver who cannot get out of the carpark due to the traffic on the roads. (A
ID:  _MJn4VJUPwE
PROMPT:  imagine a scene with all of these sounds existing together: Music, Chatter, Speech, Inside, and small room. Then describe this scene in one sentence. 

 imagine a scene with all of these sounds existing together: Music, Chatter, Speech, Inside, and small room. Then describe this scene in one sentence.
This exercise and others like it can help you get your mind read